# MongoDB Atlas Full-Text Search Retrieval with Claude

This notebook provides a step-by-step guide for using the MongoDB Atlas search tool with Claude. We will:

1. Set up the environment and imports
2. Build a search tool to query an MongoDB Atlas cluster
3. Test the search tool  
4. Create a Claude client with access to the tool 
5. Compare Claude's responses with and without access to the tool

## Imports and Configuration 

First we'll import libraries and load environment variables. This includes setting up logging so we can monitor the process.

In [1]:
import os
import sys
import dotenv
import anthropic

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import claude_retriever

# Load environment variables
dotenv.load_dotenv(dotenv_path="../.env.local")

True

In [2]:
# Import and configure logging 
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to log to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

In [3]:
mongo_connection_string = os.environ['MONGO_CONNECTION_STR']
mong_db_name = os.environ['MONG_DB_NAME']
mongo_collection_name = os.environ['MONG_COLLECTION_NAME']

# Store your data

The first step is setting up your datastore. Here, we will make use of the [Kaggle Amazon Products 2020 Dataset](https://www.kaggle.com/datasets/promptcloud/amazon-product-dataset-2020). It contains 10000 products from Amazon, including their product title, description, price, category tags, etc. For the purposes of this notebook, we've pre-processed the data to concatenate the title, description and category tags into a single "document" field and saved it locally as a JSONL with one line for each product.

We now need to transform this raw text dataset into an embedding dataset. In this notebook we will opt for the simplest possible way to do this locally:

1. We will use the [sentence-transformers](https://www.sbert.net/index.html) library, which allows us to use a lightweight model to embed our text data using only a CPU if that is all we have available.
2. We will save the text/embedding pairs on MongoDB Atlas that can be used to query and augment the custom prompts to LLM.



In [6]:
# Set up MongoDB Atlas instance and upload the data
from claude_retriever.searcher.searchtools.mongodb import MongoDBAtlasSearchTool
AMAZON_SEARCH_TOOL_DESCRIPTION = 'The search engine will search over the Amazon Product database, and return for each product its title, description, and a set of tags.'
amazon_search_tool = MongoDBAtlasSearchTool(tool_description=AMAZON_SEARCH_TOOL_DESCRIPTION,
                                            mongo_connection_string= mongo_connection_string,
                                            mongo_database= mong_db_name,
                                            mongo_collection= mongo_collection_name,
                                            query_field= "text",
                                            project_fields= {"_id":0 , "score": {"$meta": "searchScore"}, "text": 1},
                                            output_field= "text",
                                            )

In [17]:
# Load data to mongodb [Optional]
# this will add data to the mongodb collection specified and also create a dynamic index
# for more customized index, `amazon_search_tool.index_mapping` can be updated with custom mapping
amazon_search_tool.upload_data_to_mongodb("./data/amazon-products.jsonl")

In [7]:
amazon_search_tool.raw_search("fluffy toys")

[SearchResult(content='Product Name: Creative Converting 3-Count Fluffy Tissue Balls, Classic Pink\n\nAbout Product: 3-Count paper fluffy balls in Classic Pink | Balls 16-Inches in diameter | Tissue paper with flower petal-like edges | Great as a hanging decoration | Partner with Creative Converting for the best in paper decorations for holiday celebrations and theme parties\n\nCategories: Toys & Games | Party Supplies'),
 SearchResult(content='Product Name: Aurora Rattlesnake Plush, Green\n\nAbout Product: Fine plush fabric. Soft and fluffy. 13 inches in size.\n\nCategories: Toys & Games | Stuffed Animals & Plush Toys | Stuffed Animals & Teddy Bears'),
 SearchResult(content='Product Name: Amscan 225555.12 Garland, Multi Size, Blue\n\nAbout Product: 2 pieces 12\' ribbons per pack with 9 pieces fluffy tissue balls, measuring 5.5" each | Blue long ribbons with fluffy tissue balls attached, hanging decoration | Perfect for engagement party, wedding receptions and wedding anniversary parti

# Create a Search Tool for your data

We now create a Search Tool, which can take queries and return formatted relevant results. We also need to describe what the search tool will return, which Claude will read to make sure it is correctly used.

In [ ]:
from claude_retriever.searcher.searchtools.elasticsearch import ElasticsearchCloudSearchTool

AMAZON_SEARCH_TOOL_DESCRIPTION = 'The search engine will search over the Amazon Product database, and return for each product its title, description, and a set of tags.'
amazon_search_tool = ElasticsearchCloudSearchTool(tool_description=AMAZON_SEARCH_TOOL_DESCRIPTION,
                                                  elasticsearch_cloud_id=cloud_id,
                                                  elasticsearch_api_key_id=api_key_id,
                                                  elasticsearch_api_key=api_key,
                                                  elasticsearch_index=index_name)

Let's test it to see if the tool works!

In [8]:
dinos = amazon_search_tool.search("fun kids dinosaur book", n_search_results_to_use=3)
print(dinos)


<search_results>
<item index="1">
<page_content>
Product Name: Wild Republic T-Rex Plush, Dinosaur Stuffed Animal, Plush Toy, Gifts for Kids, Dinosauria 17 Inches

About Product: Bring dinosaurs to life for your child with this realistic T Rex toy that is designed to capture the detail of this King of the dinosaurs | Wild Republic Stuffed dinosaur toys are a great way for kids to play and learn at the same time | Collect each dino in our 17" Dinosaurian collection which includes the Stegosaurus, Triceratops, Velociraptor, Diplodocus, and the T Rex | Your child will enjoy this stuffed dinosaur Plush because of its realistic design and pose. The textured fabric truly brings this T-Rex to life | Great birthday gift for boys, dinosaur themed birthday parties or bedrooms. Tyrannosaurus Rex is the most popular of the stuffed dinosaurs

Categories: Toys & Games | Stuffed Animals & Plush Toys | Stuffed Animals & Teddy Bears
</page_content>
</item>
<item index="2">
<page_content>
Product Name:

# Use Claude with Retrieval

We can now simply pass this search tool to Claude to use, much in the same way a person might.

In [9]:
ANTHROPIC_SEARCH_MODEL = "claude-2"

client = claude_retriever.ClientWithRetrieval(api_key=os.environ['ANTHROPIC_API_KEY'], search_tool = amazon_search_tool)

query = "I want to get my daughter more interested in science. What kind of gifts should I get her?"
prompt = f'{anthropic.HUMAN_PROMPT} {query}{anthropic.AI_PROMPT}'

Here is the basic response to the query (no access to the tool).

In [10]:
basic_response = client.completions.create(
    prompt=prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)
print('-'*50)
print('Basic response:')
print(prompt + basic_response.completion)
print('-'*50)

2024-02-12 12:20:14,399 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
--------------------------------------------------
Basic response:


Human: I want to get my daughter more interested in science. What kind of gifts should I get her?

Assistant: Here are some science-themed gift ideas to spark a child's interest:

- Microscope or telescope set - Let them explore tiny worlds or gaze at stars. Get one suited for their age.

- Chemistry set - Look for a safe set that allows them to do basic experiments like making slime or crystals. Supervise young kids.

- Robot building kit - Kits allow them to construct then program motorized robots. Great for older kids.

- Rock/mineral collection - Include a book to help identify different specimens.

- Science kits - Various topics like oceanography, archeology, ecology. Some have kids conduct experiments.

- Books - Find ones that tie into their interests and are full of engaging experiments and acti

Now we get the same completion, but give Claude the ability to use the tool when thinking about the response.

In [11]:
augmented_response = client.completion_with_retrieval(
    query=query,
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=3,
    max_tokens_to_sample=1000)

print('-'*50)
print('Augmented response:')
print(prompt + augmented_response)
print('-'*50)

2024-02-12 12:20:32,385 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2024-02-12 12:20:32,387 - claude_retriever.client - INFO -  <thinking>
To gather helpful information to answer this query, I should search for science books, toys, kits, or other educational products targeted at children that could spark their interest in science. I'll try searching for a variety of science-related terms along with terms indicating products for children or gifts. I also want to find products with good reviews or descriptions that explain why they are engaging and educational for kids.
</thinking>

<search_query>science books for children gift
2024-02-12 12:20:32,391 - claude_retriever.client - INFO - Attempting search number 0.
2024-02-12 12:20:32,391 - claude_retriever.client - INFO - 
--------------------
Pausing stream because Claude has issued a query in <search_query> tags: <search_query>science books for children gift</search_query>
----------------

Often, you'll want finer-grained control about how exactly Claude uses the results. For this workflow we recommend "retrieve then complete".

In [12]:
relevant_search_results = client.retrieve(
    query=query,
    stop_sequences=[anthropic.HUMAN_PROMPT, 'END_OF_SEARCH'],
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=3,
    max_searches_to_try=5,
    max_tokens_to_sample=1000)

print('-'*50)
print('Relevant results:')
print(relevant_search_results)
print('-'*50)

2024-02-12 12:22:14,223 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2024-02-12 12:22:14,226 - claude_retriever.client - INFO -  <thinking>
To gather relevant information to help answer this query, I should search for science kits, books, toys, or other products aimed at getting girls more interested in science. Relevant information would include descriptions of products, recommended ages, and topics covered. I'll search broadly at first to see what's available, then refine my search as needed focusing on products for the appropriate age range that cover interesting science topics.
</thinking>

<search_query>science gifts for girls age 10
2024-02-12 12:22:14,229 - claude_retriever.client - INFO - Attempting search number 0.
2024-02-12 12:22:14,230 - claude_retriever.client - INFO - 
--------------------
Pausing stream because Claude has issued a query in <search_query> tags: <search_query>science gifts for girls age 10</search_query>
-----

In [13]:
qa_prompt = f'''{anthropic.HUMAN_PROMPT} You are a friendly product recommender. Here is a query issued by a user looking for product recommendations:

{query}

Here are a set of search results that might be helpful for answering the user's query:

{relevant_search_results}

Once again, here is the user's query:

<query>{query}</query>

Please write a response to the user that answers their query and provides them with helpful product recommendations. Feel free to use the search results above to help you write your response, or ignore them if they are not helpful.

At the end of your response, under "Products you might like:", list the top 3 product names from the search results that you think the user would most like.

Please ensure your results are in the following format:

<result>
Your response to the user's query.
</result>
<recommendations>
Products you might like:
1. Product name
2. Product name
3. Product name
</recommendations>{anthropic.AI_PROMPT}'''

response = client.completions.create(
    prompt=qa_prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)

print('-'*50)
print('Response:')
print(response.completion)
print('-'*50)

2024-02-12 12:23:10,739 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
--------------------------------------------------
Response:
 Here is a response with product recommendations for getting your daughter more interested in science:

<result>
Based on your interest in getting your daughter more interested in science, I would recommend getting her an engaging science kit or set that allows her to conduct hands-on experiments and activities. The 4M KidzLabs Fingerprint Kit, Hey! Play! Kids Science Kit, and The Learning Journey Techno Kids Stack & Spin Gears Super Set all seem like great options that could spark her curiosity and get her excited about scientific discovery. These kits cover diverse science topics like forensics, chemistry, and engineering, allowing her to explore what interests her most. They provide hours of educational play and will help strengthen important skills like critical thinking and problem solving. Best of all, the